<a id='afinando-un-trabajo-de-spark'></a>
## Ajuste fino de un trabajo de Spark

Antes de empezar, ten en cuenta que toda esta sección está escrita exclusivamente en base a la experiencia. Puede diferir con los casos de uso, pero le ayudará a tener una mejor comprensión de lo que debe buscar, o actuar como una guía para lograr su objetivo.

>El ajuste del rendimiento de Spark se refiere al proceso de ajustar la configuración para registrar la memoria, los núcleos y las instancias utilizadas por el sistema. Este proceso garantiza que Spark tenga un rendimiento impecable y también evita el cuello de botella de los recursos en Spark.

Teniendo en cuenta que estás utilizando Amazon EMR para ejecutar tus trabajos de Spark, hay tres aspectos que debes cuidar:
1. Dimensionamiento de EMR
2. Configuraciones de Spark
3. Ajuste de trabajos


<a id='emr-sizing'></a>
### EMR/Dataproc Sizing

Dimensionar tu EMR es extremadamente importante, ya que esto afecta a la eficiencia de tus trabajos en Spark. Aparte del factor del coste, el número máximo de nodos y de memoria que su trabajo puede utilizar se decidirá por esto. Si se hace trabajar un EMR con altas especificaciones, obviamente significa que se está pagando más por él, por lo que lo ideal es utilizarlo al máximo. Estas son las pautas que sigo para asegurarme de que el EMR está correctamente dimensionado:

1. Tamaño de los datos de entrada (incluye todos los datos de entrada) en el disco.
2. Si los trabajos tienen transformaciones o simplemente pasan directamente. Evalúe las uniones y las uniones complejas involucradas.
3. Tamaño de los datos de salida en el disco.

Mire los criterios anteriores contra la memoria que necesita procesar y el espacio en disco que necesitaría. Comience con una configuración pequeña y siga agregando nodos para llegar a una configuración óptima. En caso de que se pregunte sobre el factor *Tiempo de ejecución frente a la configuración de EMR*, comprenda que está bien que un trabajo se ejecute durante más tiempo, en lugar de agregar más recursos al clúster. Por ejemplo, está bien ejecutar un trabajo durante 40 minutos en un clúster de 5 nodos, en lugar de ejecutar un trabajo en 10 minutos en un clúster de 15 nodos.


Otra cosa que debe saber sobre los EMR son los diferentes tipos de instancias EC2 proporcionados por Amazon. Hablaré brevemente sobre ellos, pero le recomiendo que lea más sobre ellos en la [documentación oficial] (https://aws.amazon.com/ec2/instance-types/). Hay 5 tipos de clases de instancia. Según el trabajo que desee ejecutar, puede decidir cuál usar:

>Clase de instancia | Descripción
>--- | ---
>Propósito general | Equilibrio de recursos informáticos, de memoria y de red
> Computación optimizada | Ideal para aplicaciones informáticas que se benefician de procesadores de alto rendimiento
>Memoria optimizada | Diseñado para ofrecer un rendimiento rápido para cargas de trabajo que procesan grandes conjuntos de datos en la memoria
>Almacenamiento optimizado | Para cargas de trabajo que requieren un alto acceso de lectura y escritura secuencial a conjuntos de datos muy grandes en el almacenamiento local
>Instancias de GPU | Use aceleradores de hardware, o coprocesadores, para realizar funciones de alta demanda, de manera más eficiente de lo que es posible en el software que se ejecuta en las CPU.

La configuración (memoria, almacenamiento, CPU, rendimiento de la red) diferirá según la clase de instancia que elija.<br>
Para ayudar a hacer la vida más fácil, esto es lo que hago cuando me encuentro en un aprieto sobre cuál elegir: <br>
 1. Visite [ec2instances](https://www.ec2instances.info/) [gcpInstances](https://gcpinstances.doit-intl.com/)
 2. Elija las instancias EC2 en cuestión
 3. Haga clic en comparar seleccionado

¡Esto lo ayudará fácilmente a comprender en qué se está metiendo y, por lo tanto, lo ayudará a tomar la mejor decisión! El sitio fue creado por [Garret Heaton](https://github.com/powdahound)(fundador de Swoot).

<a id='spark-configurations'></a>
### Spark Configurations

Hay un montón de [configuraciones] (https://spark.apache.org/docs/latest/configuration.html) que puede modificar cuando se trata de Spark. Aquí, anotaré algunas de las configuraciones que uso, que me han funcionado bien.

#### Job Scheduling

Cuando envíe su trabajo en un clúster, se entregará a Spark Schedulers, que es responsable de materializar un plan lógico para su trabajo. Hay dos tipos de [programación de trabajos](https://spark.apache.org/docs/latest/job-scheduling.html):
1. FIFO<br>
De forma predeterminada, el programador de Spark ejecuta trabajos en modo FIFO. Cada trabajo se divide en etapas (por ejemplo, mapear y reducir fases), y el primer trabajo tiene prioridad en todos los recursos disponibles mientras que sus etapas tienen tareas para iniciar, luego el segundo trabajo tiene prioridad, etc. no es necesario usar todo el clúster, los trabajos posteriores pueden comenzar a ejecutarse de inmediato, pero si los trabajos al principio de la cola son grandes, es posible que los trabajos posteriores se retrasen significativamente.
2. FAIR<br>
El programador FAIR admite la agrupación de trabajos en grupos y la configuración de diferentes opciones de programación (por ejemplo, peso) para cada grupo. Esto puede ser útil para crear un grupo de alta prioridad para trabajos más importantes, por ejemplo, o para agrupar los trabajos de cada usuario y otorgarles partes iguales a los usuarios, independientemente de cuántos trabajos simultáneos tengan, en lugar de darles partes iguales a los trabajos. Este enfoque sigue el modelo de Hadoop Fair Scheduler.

> Personalmente prefiero usar el modo FAIR, y esto se puede configurar agregando `.config("spark.scheduler.mode", "FAIR")` cuando creas tu SparkSession.

#### Serializer

Tenemos dos tipos de [serializadores](https://spark.apache.org/docs/latest/tuning.html#data-serialization) disponibles:
1. Serialización de Java
2. Serialización de Kryo

Kryo es significativamente más rápido y más compacto que la serialización de Java (a menudo hasta 10x), pero no es compatible con todos los tipos serializables y requiere que registre las clases que usará en el programa con anticipación para obtener el mejor rendimiento.

La serialización de Java se usa de forma predeterminada porque si tiene una clase personalizada que amplía Serializable, se puede usar fácilmente. También puede controlar el rendimiento de su serialización más de cerca extendiendo java.io.Externalizable

> La recomendación general es usar Kyro como serializador siempre que sea posible, ya que conduce a tamaños mucho más pequeños que la serialización de Java. Se puede agregar usando `.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")` cuando crea su SparkSession.

#### Shuffle Behaviour

Por lo general, es una buena idea comprimir el archivo de salida después de la fase de mapeo. La propiedad `spark.shuffle.compress` decide si hacer la compresión o no. La compresión utilizada es `spark.io.compression.codec`.

> La propiedad se puede agregar usando `.config("spark.shuffle.compress", "true")` cuando crea su SparkSession.

#### Compression and Serialization

Hay 4 códecs predeterminados que Spark proporciona para comprimir datos internos, como particiones RDD, registro de eventos, variables de transmisión y salidas aleatorias. Están:

1. lz4
2. lzf
3. snappy
4. zstd

> La decisión sobre cuál usar se basa en el caso de uso. Generalmente uso la compresión `snappy`. Google creó Snappy porque necesitaba algo que ofreciera una compresión muy rápida a expensas del tamaño final. Snappy es rápido, estable y gratuito, pero aumenta el tamaño más que los otros códecs. Al mismo tiempo, dado que los costos de cómputo serán menores, parece una compensación equilibrada. La propiedad se puede agregar usando `.config("spark.io.compression.codec", "snappy")` cuando crea su SparkSession.

Esta [sesión](https://databricks.com/session/best-practice-of-compression-decompression-codes-in-apache-spark) explica las mejores prácticas de compresión/descompresión de códigos en Apache Spark. Les recomiendo que le echen un vistazo antes de tomar una decisión.

#### Scheduling

La propiedad `spark.speculation` realiza la ejecución especulativa de tareas. Esto significa que si una o más tareas se ejecutan lentamente en una etapa, se volverán a iniciar. La ejecución especulativa no detendrá la tarea de ejecución lenta, pero inicia la nueva tarea en paralelo.

> Usualmente deshabilito esta opción agregando `.config("spark.speculation", "false") ` cuando creo SparkSession.

#### Application Properties

Existen principalmente dos propiedades de la aplicación que debe conocer:

1. spark.driver.memoryOverhead: la cantidad de memoria fuera del montón que se asignará por controlador en modo de clúster, en MiB, a menos que se especifique lo contrario. Esta es la memoria que representa cosas como los gastos generales de VM, cadenas internas, otros gastos generales nativos, etc. Esto tiende a crecer con el tamaño del contenedor (típicamente 6-10%). Esta opción actualmente es compatible con YARN y Kubernetes.

2. spark.executor.memoryOverhead: la cantidad de memoria fuera del montón que se asignará por ejecutor, en MiB, a menos que se especifique lo contrario. Esta es la memoria que tiene en cuenta cosas como gastos generales de VM, cadenas internas, otros gastos generales nativos, etc. Esto tiende a crecer con el tamaño del ejecutor (típicamente 6-10%). Esta opción actualmente es compatible con YARN y Kubernetes.

> Si alguna vez se enfrenta a un problema como `Contenedor eliminado por YARN por exceder los límites de memoria`, sepa que se debe a que no ha especificado suficiente sobrecarga de memoria para que su trabajo se ejecute con éxito. El valor predeterminado para Overhead es 10% de la memoria disponible

<a id='job-tuning'></a>
### Job Tuning

Además del ajuste de EMR y Spark, hay otra forma de abordar las optimizaciones, y es ajustando su propio trabajo para producir resultados de manera eficiente. Repasaré algunas de esas técnicas que te ayudarán a lograrlo. La [Guía de programación de Spark](https://spark.apache.org/docs/2.1.1/programming-guide.html) habla más sobre estos conceptos en detalle. Si prefieren ver un video en lugar de leer, les recomiendo [A Deep Dive into Proper Optimization for Spark Jobs](https://youtu.be/daXEp4HmS-E) de Daniel Tomes de Databricks, que encontré realmente útil e informativo. !

#### Broadcast Joins (Broadcast Hash Join)

Para algunos trabajos, la eficiencia se puede aumentar almacenándolos en memoria caché. Broadcast Hash Join (BHJ) es una técnica de este tipo que lo ayudará a optimizar las consultas de unión cuando el tamaño de un lado de los datos es bajo.
>Las uniones de BroadCast son las más rápidas, pero el inconveniente es que consumirá más memoria tanto en el ejecutor como en el controlador.

Los siguientes pasos brindan un adelanto de cómo funciona, lo que lo ayudará a comprender los casos de uso en los que se puede usar:<br>
1. El archivo de entrada (el más pequeño de las dos tablas) que se transmitirá es leído por los ejecutores en paralelo en su memoria de trabajo.
2. Todos los datos de los ejecutores se recopilan en el controlador (por lo tanto, la necesidad de mayor memoria en el controlador).
3. Luego, el controlador transmite el conjunto de datos combinado (copia completa) a cada ejecutor.
4. El tamaño del conjunto de datos transmitido podría ser varias (10-20+) veces mayor que la entrada en la memoria debido a factores como la deserialización.
5. Los ejecutores terminarán almacenando las partes que leyeron primero, y también la copia completa, lo que genera un alto requerimiento de memoria.


<center><img src="https://media-exp2.licdn.com/dms/image/C5612AQGo4w2cEWUDCg/article-inline_image-shrink_1000_1488/0/1554384116825?e=1662595200&v=beta&t=TQxovk8O4_rW74QJMKvDFQcchmVz4mLouD9fR1AfR1c">

#### Spark Partitions

Una partición en Spark es un fragmento atómico de datos (división lógica de datos) almacenado en un nodo del clúster. Las particiones son las unidades básicas de paralelismo en Spark. Tener un número demasiado grande de particiones o muy pocas no es una solución ideal. La cantidad de particiones en Spark debe decidirse en función de la configuración del clúster y los requisitos de la aplicación. Aumentar el número de particiones hará que cada partición tenga menos datos o no tenga datos. En general, la partición de Spark se puede dividir de tres maneras:

1. Input
2. Shuffle
3. Output

##### Input

Spark generalmente hace un buen trabajo al calcular la configuración ideal para este, excepto en casos muy particulares. Es recomendable utilizarlo por defecto a menos que:
1. Aumentar el paralelismo
2. Datos muy anidados
3. Generación de datos (explotar)
4. La fuente no es óptima
5. Estás usando UDF

`spark.sql.files.maxpartitionBytes`: esta propiedad indica el número máximo de bytes para empaquetar en una sola partición al leer archivos (predeterminado 128 MB). Use esto para aumentar el paralelismo en la lectura de datos de entrada. Por ejemplo, si tiene más núcleos, puede aumentar la cantidad de tareas paralelas, lo que garantizará el uso de todos los núcleos del clúster y aumentará la velocidad de la tarea.

##### Shuffle

Una de las principales razones por las que la mayoría de los trabajos se retrasan en el rendimiento es, la mayoría de las veces, porque las particiones aleatorias no cuentan correctamente. De forma predeterminada, el valor se establece en 200. En casi todas las situaciones, esto no es lo ideal. Si está tratando con una capacidad aleatoria de menos de 20 GB, 200 está bien, pero de lo contrario, debe cambiarse. En la mayoría de los casos, puede usar la siguiente ecuación para encontrar el valor correcto:
>`Recuento de particiones = Datos de entrada de etapa/Tamaño de destino` donde <br>
`Etapa aleatoria más grande (Tamaño de destino) < 200 MB/partición` en la mayoría de los casos.<br>
La propiedad `spark.sql.shuffle.partitions` se utiliza para establecer el valor ideal de recuento de particiones.

Si alguna vez nota ese tamaño objetivo en el rango de TB, hay algo terriblemente mal y es posible que desee volver a cambiarlo a 200 o recalcularlo. Las particiones aleatorias se pueden configurar para cada acción (no transformación) en el script de Spark.

Usemos un ejemplo para explicar este escenario: <br>
Suponga que la entrada de etapa aleatoria = 210 GB. <br>
Recuento de particiones = Datos de entrada de etapa/Tamaño de destino = 210 000 MB/200 MB = 1050. <br>
Como puede ver, mis particiones aleatorias deberían ser 1050, no 200.

Pero, si su clúster tiene 2000 núcleos, configure sus particiones aleatorias en 2000.
>En un clúster grande que se ocupa de un gran trabajo de datos, nunca establezca sus particiones aleatorias en menos que su recuento total de núcleos.

##### Output

Hay diferentes métodos para escribir los datos. Puede controlar el tamaño, la composición, la cantidad de archivos en la salida e incluso la cantidad de registros en cada archivo mientras escribe los datos. Mientras escribe los datos, puede aumentar el paralelismo, asegurando así que utiliza todos los recursos que tiene. Pero este enfoque conduciría a una mayor cantidad de archivos más pequeños. Por lo general, esto no es un problema, pero si desea archivos más grandes, deberá usar una de las técnicas de compactación, preferiblemente en un clúster con una configuración menor. Hay varias formas de cambiar la composición de la salida. Tenga en cuenta estos dos sobre la composición:
1. Coalesce: Use esto para reducir el número de particiones.
2. Repartición: use esto muy raramente y nunca para reducir la cantidad de particiones<br>
    una. Separador de rangos: divide los datos en función de algún orden ordenado O un conjunto de rangos ordenados de claves. <br>
    b. Hash Partitioner: distribuye los datos en la partición en función del valor de la clave. La partición hash puede sesgar los datos distribuidos.

<a id='mejores prácticas'></a>
### Mejores prácticas

Trate de incorporar estos a sus hábitos de codificación para un mejor rendimiento:
1.   1. No utilice NOT IN, sino NOT EXISTS.
2.   Elimine los recuentos, los recuentos distintos (utilice approxCountDIstinct).
3.   Elimine los duplicados antes de tiempo.
4.   Prefiera siempre las funciones SQL sobre PandasUDF.
5.   Utilizar eficazmente las particiones. 
6.   Intentar que la utilización del clúster sea al menos del 70%.



![Spark Image](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Apache_Spark_logo.svg/1200px-Apache_Spark_logo.svg.png)
# Qué es Apache Spark ?

Apache Spark es un motor de cálculo "unificado" y un conjunto de "bibliotecas" para el "procesamiento paralelo de datos" en clusters de ordenadores.

En lo que respecta a la velocidad, Spark "amplía" el popular modelo "MapReduce" para soportar de forma eficiente más tipos de cálculos, incluidas las consultas interactivas y el procesamiento de flujos. Una de las principales características que ofrece Spark en cuanto a velocidad es la capacidad de ejecutar cálculos en memoria, pero el sistema también es más eficiente que MapReduce para aplicaciones complejas que se ejecutan en disco.

<center><img src="https://i.ibb.co/4tKqXdm/Spark.png" alt= "Spark"></img>
<br>
<em>Simple illustration of all that Spark has to offer an end user<em></center>
    



## La filosofía de Apache Spark
*Desglosemos nuestra descripción de Apache Spark -un motor de computación unificado y un conjunto de bibliotecas para big data- en sus componentes clave.*

## Una pila unificada
El proyecto Spark contiene múltiples componentes estrechamente integrados. 
- En su núcleo, Spark es un "motor computacional" que se encarga de programar, distribuir y supervisar las aplicaciones
- Impulsa múltiples componentes de nivel superior especializados en diversas cargas de trabajo, como SQL o el aprendizaje automático
- Todas las bibliotecas y componentes de nivel superior de la pila se benefician de las mejoras en las capas inferiores.
    > Por ejemplo, cuando el motor central de Spark añade una optimización, las bibliotecas de SQL y de aprendizaje automático también se aceleran. 
- Los costes asociados a la ejecución de la pila se reducen al mínimo, ya que en lugar de ejecutar entre 5 y 10 procesos independientes, una organización sólo necesita ejecutar uno.
- Una de las mayores ventajas de la integración estrecha es la capacidad de construir aplicaciones que combinan sin problemas diferentes modelos de procesamiento.
    > Por ejemplo, en Spark se puede escribir una aplicación que utilice el aprendizaje automático para clasificar los datos en tiempo real a medida que se reciben de fuentes de flujo. Simultáneamente, los analistas pueden consultar los datos resultantes, también en tiempo real, a través de SQL

## Computing engine

Al mismo tiempo que Spark se esfuerza por la unificación, limita cuidadosamente su alcance a un motor de computación. 
- Podemos utilizar Spark con una amplia variedad de sistemas de almacenamiento persistente, incluyendo sistemas de almacenamiento en la nube como Azure Storage, Amazon S3 y Dataproc, sistemas de archivos distribuidos como Apache Hadoop, almacenes de valores clave como Apache Cassandra y buses de mensajes como Apache Kafka. 
- Spark no almacena los datos a largo plazo por sí mismo, ni favorece uno sobre otro.
- Los datos son computacionalmente caros de mover, por lo que Spark se centra en realizar cálculos sobre los datos, sin importar dónde residan.

## Libraries

El último componente de Spark son sus librerías, que se basan en su diseño como motor unificado para proporcionar una API unificada para las tareas comunes de análisis de datos. Las librerías estándar de Spark son en realidad el grueso de los proyectos de código abierto
- Spark incluye librerías para SQL y datos estructurados (Spark SQL), aprendizaje automático (MLlib), procesamiento de flujos (Spark Streaming y la más reciente Structured Streaming) y análisis de gráficos (GraphX). 
- Además de estas librerías, hay cientos de librerías externas de código abierto que van desde conectores para varios sistemas de almacenamiento hasta algoritmos de aprendizaje automático.

## Arquitectura básica de Spark

Las máquinas solas no tienen suficiente potencia y recursos para realizar cálculos sobre grandes cantidades de información. Un `cluster`, o grupo, de ordenadores, agrupa los recursos de muchas máquinas juntas, dándonos la capacidad de utilizar todos los recursos acumulados como si fueran un solo ordenador. ero, un grupo de máquinas por sí solo no es potente, se necesita un marco para coordinar el trabajo entre ellas. Spark hace precisamente eso, gestionar y coordinar la ejecución de
tareas sobre los datos en un clúster de ordenadores.

El clúster de máquinas que Spark utilizará para ejecutar las tareas es gestionado por un gestor de clústeres como
`Spark's standalone cluster manager`, `YARN`, o `Mesos`. [YARN](https://es.wikipedia.org/wiki/Yarn_(Facebook))

## Spark Applications

**Las aplicaciones Spark consisten en un proceso `driver` y un conjunto de procesos `ejecutores`.

- El proceso `driver` es el corazón de una aplicación Spark, su función main(), se sitúa en un nodo del cluster, y es responsable de tres cosas
    - mantener la información sobre la aplicación Spark
    - responder a un programa o entrada del usuario
    - Analizar, distribuir y programar el trabajo entre los ejecutores
- Los ejecutores son responsables de llevar a cabo el trabajo que el controlador les asigna. Esto significa que cada ejecutor es responsable de sólo dos cosas
    - ejecutar el código asignado por el controlador,
    - informar del estado del cálculo en ese ejecutor al nodo controlador
    
    <center><img src="https://i.ibb.co/BBKv55G/Spark-Application.png" alt="Spark-Application" border="0"><br>En esta ilustración vemos a la izquierda nuestro controlador y a la derecha los cuatro ejecutores de la derecha. En este diagrama se elimina el concepto de nodos del cluster. El usuario puede especificar cuántos ejecutores deben caer en cada nodo a través de las configuraciones.
    <em></em></center>
    <br>
    
     **_NOTA:_** Spark, además de su modo cluster, también tiene un modo local. El controlador y los ejecutores son simplemente procesos, lo que significa que pueden vivir en la misma máquina o en máquinas diferentes. En modo local, el controlador y los ejecutores se ejecutan (como hilos) en su ordenador individual en lugar de un clúster.

## APIs de lenguaje de Spark

Las APIs de lenguaje de Spark permiten ejecutar el código de Spark utilizando varios lenguajes de programación.

- Scala
- Java
- Python
- SQL
- R

Hay un objeto `SparkSession` disponible para el usuario, que es el punto de entrada para ejecutar el código de Spark. Cuando se utiliza Spark desde Python o R, no se escriben instrucciones explícitas en la JVM; en su lugar, se escribe código Python y R que Spark traduce en código que luego puede ejecutar en las JVMs ejecutoras.

<center><img src="https://i.ibb.co/8B1kjnq/Language-API.png" alt="Language-API" border="0">
<em><br>The relationship between the SparkSession and Spark’s Language API</em></center>
<br>
Spark tiene dos conjuntos fundamentales de APIs: las APIs de bajo nivel `no estructuradas`, y las APIs de alto nivel `estructuradas`.
<center><img src="https://i.ibb.co/PW8KCG2/Spark-APIs.png" alt="Spark-APIs" border="0">

# APIs de bajo nivel no estructuradas

En este cuaderno discutiremos el concepto fundamental más antiguo de spark llamado *RDDs(Resilient distributed
datasets)*.

<br> 
Para entender realmente cómo funciona Spark, `hay que entender la esencia de los RDDs`.Estos proporcionan una base extremadamente sólida sobre la que se construyen otras abstracciones. A partir de Spark 2.0, los usuarios de Spark tendrán menos necesidades de interactuar directamente con los RDD, pero es esencial tener un modelo mental sólido de cómo funcionan los RDD. `En pocas palabras, Spark gira en torno al concepto de RDD`.

## Re-Introduction to RDDs

Un RDD en Spark es simplemente una colección distribuida inmutable de objetos. Cada uno se divide en múltiples particiones, que pueden ser calculadas en diferentes nodos del clúster.
<br>
Los RDDs son `inmutables`, `tolerantes a fallos`, `estructuras de datos paralelas` que permiten a los usuarios persistir explícitamente los resultados intermedios `en memoria`, controlar su partición para optimizar la colocación de los datos, y `manipularlos` utilizando un rico conjunto de `operadores`.


## Inmutable

Los RDDs están diseñados para ser inmutables, lo que significa que usted `no puede` modificar específicamente una fila particular en el conjunto de datos representado por ese RDD. Puedes llamar a una de las operaciones disponibles para manipular las filas del RDD de la forma que quieras, pero esa operación `devolverá un nuevo RDD`. El `RDD básico permanecerá sin cambios`, y el nuevo RDD
contendrá los datos de la forma que usted desee. *Spark aprovecha la Inmutabilidad para proporcionar eficientemente la capacidad de tolerancia a fallos.* 



## Tolerancia a los fallos

La capacidad de procesar múltiples conjuntos de datos en paralelo suele requerir un clúster de máquinas para alojar y ejecutar la lógica computacional. Si una o más máquinas mueren debido a circunstancias inesperadas, ¿qué pasa con los datos en esas máquinas?  Spark se encarga automáticamente de gestionar el fallo en nombre de sus usuarios reconstruyendo la parte que ha fallado utilizando la información del linaje.

## Estructuras de Datos Paralelas

Suponga que tiene una gran cantidad de datos y necesita procesar todas y cada una de las filas del conjunto de datos. Una solución sería iterar sobre cada fila y procesarla una por una. Pero eso sería muy lento. Así que, en lugar de eso, dividiremos la enorme cantidad de datos en trozos más pequeños. Cada trozo contiene una colección de filas, y todos los trozos se procesan en paralelo. De ahí viene la expresión "estructuras de datos paralelas".


## Computación en memoria

La idea de acelerar el cálculo de grandes conjuntos de datos que residen en discos de forma paralela utilizando un clúster de máquinas fue introducida por un documento de MapReduce de Google. RDD amplía los límites de la velocidad introduciendo una idea novedosa, que es la capacidad de realizar cálculos distribuidos en memoria.

## Operaciones con RDDs

Los RDDs proporcionan un rico conjunto de operaciones de procesamiento de datos comúnmente necesarias. Incluyen la capacidad de realizar transformaciones de datos, filtrado, agrupación, unión, agregación, ordenación y recuento.

Cada fila de un conjunto de datos se representa como un objeto Java, y la estructura de este objeto Java es opaca para Spark. El usuario de RDD tiene un control total sobre cómo manipular este objeto Java. Esta flexibilidad viene con muchas responsabilidades, lo que significa que algunas de las operaciones comúnmente necesarias, como la media de cálculo, tendrán que ser hechas a mano. Las abstracciones de nivel superior, como el componente SQL de Spark, proporcionarán esta funcionalidad fuera de la caja.

***Las operaciones del RDD se clasifican en dos tipos: `transformaciones` y `acciones`***

| Tipo de operación: Evaluación: Valor devuelto.
|--|--|--|
| Transformación: perezoso, otro RDD.
| Acción: Ansioso: Algún resultado o escribir el resultado en el disco.

Las operaciones de transformación se evalúan de forma perezosa, lo que significa que Spark retrasará las evaluaciones de las operaciones invocadas hasta que se realice una acción. En otras palabras, las operaciones de transformación se limitan a registrar la lógica de transformación especificada y la aplicarán en un momento posterior. Por otro lado, la invocación de una operación de acción desencadenará la evaluación de todas las transformaciones que la precedieron, y devolverá algún resultado al controlador o escribirá datos en un sistema de almacenamiento, como HDFS o el sistema de archivos local.

## Initialising Spark

In [1]:
!pip install pyspark pandas

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setMaster("local").setAppName("Tutorial").set("spark.files.overwrite", "true")
sc = SparkContext(conf = conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/27 20:57:20 INFO SparkEnv: Registering MapOutputTracker
25/05/27 20:57:20 INFO SparkEnv: Registering BlockManagerMaster
25/05/27 20:57:20 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/05/27 20:57:20 INFO SparkEnv: Registering OutputCommitCoordinator


## Creación de RDDs

**Hay dos maneras de crear RDDs:**

**La primera forma de crear un RDD es paralelizando un objeto de Python, es decir, convirtiéndolo en un conjunto de datos distribuido que puede ser operado en paralelo.

In [4]:
stringList = ["Spark is awesome","Spark is cool"]
stringRDD = sc.parallelize(stringList)

In [5]:
stringRDD

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [6]:
stringRDD.collect()

['Spark is awesome', 'Spark is cool']

**La segunda forma de crear un RDD es leer un conjunto de datos de un sistema de almacenamiento, que puede ser un sistema de archivos del ordenador local, HDFS, Cassandra, Amazon S3, etc.

In [7]:
from pyspark import SparkFiles

data_file_https_url = "https://raw.githubusercontent.com/databricks/spark-training/refs/heads/master/data/movielens/medium/ratings.dat"
sc.addFile(data_file_https_url)
filePath  = 'file://' +SparkFiles.get('ratings.dat')

ratings = sc.textFile(filePath) 
ratings = ratings.map(lambda line: line.split('::')) 

In [8]:
print(ratings.first())

['1', '1193', '5', '978300760']


In [9]:
ratings.take(5)

[['1', '1193', '5', '978300760'],
 ['1', '661', '3', '978302109'],
 ['1', '914', '3', '978301968'],
 ['1', '3408', '4', '978300275'],
 ['1', '2355', '5', '978824291']]

In [10]:
ratings.count()

1000209

En este ejemplo en particular teníamos 1M de filas llamando a .collect() no tomó mucho tiempo pero si tu RDD contiene 100 mil millones de filas, entonces no es una buena idea invocar la acción collect porque el programa del driver muy probablemente no tiene suficiente memoria para mantener todas esas filas. Como resultado, el controlador probablemente se encontrará con un error de falta de memoria y su aplicación Spark o shell morirá. Esta acción se utiliza normalmente una vez que el RDD se ha filtrado a un tamaño más pequeño que puede encajar en el tamaño de la memoria del programa controlador. 

## Transformaciones

Las transformaciones son operaciones sobre RDDs que devuelven un nuevo RDD. Los RDDs transformados se calculan de forma perezosa, sólo cuando se
se utilizan en una acción.

La siguiente tabla describe las transformaciones más utilizadas.


<table>
<tbody><tr><th style="width:25%">Transformation</th><th>Meaning</th></tr>
<tr>
  <td> <b>map</b>(<i>func</i>) </td>
  <td> Return a new distributed dataset formed by passing each element of the source through a function <i>func</i>. </td>
</tr>
<tr>
  <td> <b>filter</b>(<i>func</i>) </td>
  <td> Return a new dataset formed by selecting those elements of the source on which <i>func</i> returns true. </td>
</tr>
<tr>
  <td> <b>flatMap</b>(<i>func</i>) </td>
  <td> Similar to map, but each input item can be mapped to 0 or more output items (so <i>func</i> should return a Seq rather than a single item). </td>
</tr>
<tr>
  <td> <b>mapPartitions</b>(<i>func</i>) <a name="MapPartLink"></a> </td>
  <td> Similar to map, but runs separately on each partition (block) of the RDD, so <i>func</i> must be of type
    Iterator&lt;T&gt; =&gt; Iterator&lt;U&gt; when running on an RDD of type T. </td>
</tr>
<tr>
  <td> <b>mapPartitionsWithIndex</b>(<i>func</i>) </td>
  <td> Similar to mapPartitions, but also provides <i>func</i> with an integer value representing the index of
  the partition, so <i>func</i> must be of type (Int, Iterator&lt;T&gt;) =&gt; Iterator&lt;U&gt; when running on an RDD of type T.
  </td>
</tr>
<tr>
  <td> <b>sample</b>(<i>withReplacement</i>, <i>fraction</i>, <i>seed</i>) </td>
  <td> Sample a fraction <i>fraction</i> of the data, with or without replacement, using a given random number generator seed. </td>
</tr>
<tr>
  <td> <b>union</b>(<i>otherDataset</i>) </td>
  <td> Return a new dataset that contains the union of the elements in the source dataset and the argument. </td>
</tr>
<tr>
  <td> <b>intersection</b>(<i>otherDataset</i>) </td>
  <td> Return a new RDD that contains the intersection of elements in the source dataset and the argument. </td>
</tr>
<tr>
  <td> <b>distinct</b>([<i>numPartitions</i>])) </td>
  <td> Return a new dataset that contains the distinct elements of the source dataset.</td>
</tr>
<tr>
  <td> <b>groupByKey</b>([<i>numPartitions</i>]) <a name="GroupByLink"></a> </td>
  <td> When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable&lt;V&gt;) pairs. <br>
    <b>Note:</b> If you are grouping in order to perform an aggregation (such as a sum or
      average) over each key, using <code>reduceByKey</code> or <code>aggregateByKey</code> will yield much better
      performance.
    <br>
    <b>Note:</b> By default, the level of parallelism in the output depends on the number of partitions of the parent RDD.
      You can pass an optional <code>numPartitions</code> argument to set a different number of tasks.
  </td>
</tr>
<tr>
  <td> <b>reduceByKey</b>(<i>func</i>, [<i>numPartitions</i>]) <a name="ReduceByLink"></a> </td>
  <td> When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function <i>func</i>, which must be of type (V,V) =&gt; V. Like in <code>groupByKey</code>, the number of reduce tasks is configurable through an optional second argument. </td>
</tr>
<tr>
  <td> <b>aggregateByKey</b>(<i>zeroValue</i>)(<i>seqOp</i>, <i>combOp</i>, [<i>numPartitions</i>]) <a name="AggregateByLink"></a> </td>
  <td> When called on a dataset of (K, V) pairs, returns a dataset of (K, U) pairs where the values for each key are aggregated using the given combine functions and a neutral "zero" value. Allows an aggregated value type that is different than the input value type, while avoiding unnecessary allocations. Like in <code>groupByKey</code>, the number of reduce tasks is configurable through an optional second argument. </td>
</tr>
<tr>
  <td> <b>sortByKey</b>([<i>ascending</i>], [<i>numPartitions</i>]) <a name="SortByLink"></a> </td>
  <td> When called on a dataset of (K, V) pairs where K implements Ordered, returns a dataset of (K, V) pairs sorted by keys in ascending or descending order, as specified in the boolean <code>ascending</code> argument.</td>
</tr>
<tr>
  <td> <b>join</b>(<i>otherDataset</i>, [<i>numPartitions</i>]) <a name="JoinLink"></a> </td>
  <td> When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (V, W)) pairs with all pairs of elements for each key.
    Outer joins are supported through <code>leftOuterJoin</code>, <code>rightOuterJoin</code>, and <code>fullOuterJoin</code>.
  </td>
</tr>
<tr>
  <td> <b>cogroup</b>(<i>otherDataset</i>, [<i>numPartitions</i>]) <a name="CogroupLink"></a> </td>
  <td> When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (Iterable&lt;V&gt;, Iterable&lt;W&gt;)) tuples. This operation is also called <code>groupWith</code>. </td>
</tr>
<tr>
  <td> <b>cartesian</b>(<i>otherDataset</i>) </td>
  <td> When called on datasets of types T and U, returns a dataset of (T, U) pairs (all pairs of elements). </td>
</tr>
<tr>
  <td> <b>pipe</b>(<i>command</i>, <i>[envVars]</i>) </td>
  <td> Pipe each partition of the RDD through a shell command, e.g. a Perl or bash script. RDD elements are written to the
    process's stdin and lines output to its stdout are returned as an RDD of strings. </td>
</tr>
<tr>
  <td> <b>coalesce</b>(<i>numPartitions</i>) <a name="CoalesceLink"></a> </td>
  <td> Decrease the number of partitions in the RDD to numPartitions. Useful for running operations more efficiently
    after filtering down a large dataset. </td>
</tr>
<tr>
  <td> <b>repartition</b>(<i>numPartitions</i>) </td>
  <td> Reshuffle the data in the RDD randomly to create either more or fewer partitions and balance it across them.
    This always shuffles all data over the network. <a name="RepartitionLink"></a></td>
</tr>
<tr>
  <td> <b>repartitionAndSortWithinPartitions</b>(<i>partitioner</i>) <a name="Repartition2Link"></a></td>
  <td> Repartition the RDD according to the given partitioner and, within each resulting partition,
  sort records by their keys. This is more efficient than calling <code>repartition</code> and then sorting within
  each partition because it can push the sorting down into the shuffle machinery. </td>
</tr>
</tbody></table>

## Ejemplos de transformación

### Transformación del mapa

*Devuelve un nuevo RDD aplicando una función a cada elemento de este RDD*

In [11]:
stringRDD_uppercase= stringRDD.map(lambda x: x.upper())
stringRDD_uppercase.collect()

['SPARK IS AWESOME', 'SPARK IS COOL']

In [12]:
def alternate_char_upper(text):
    new_text= []
    for i, character in enumerate(text):
        if i % 2 == 0:
            new_text.append(character.upper())
        else:
            new_text.append(character)
    return ''.join(new_text)
stringRDD_alternate_uppercase= stringRDD.map(alternate_char_upper)
stringRDD_alternate_uppercase.collect()

['SpArK Is aWeSoMe', 'SpArK Is cOoL']

### Flat Map Transfermation

*Devuelve un nuevo RDD aplicando primero una función a todos los elementos de este RDD, y luego aplanando los resultados*.

In [13]:
flatMap_Split= stringRDD.flatMap(lambda x: x.split(" "))
flatMap_Split.collect()

['Spark', 'is', 'awesome', 'Spark', 'is', 'cool']

### Diferencia entre Map y FlatMap 

In [14]:
print("Split using Map transformation:")
map_Split= stringRDD.map(lambda x: x.split(" "))
map_Split.collect()

Split using Map transformation:


[['Spark', 'is', 'awesome'], ['Spark', 'is', 'cool']]

In [15]:
print("Split using FlatMap transformation:")
flatMap_Split.collect()

Split using FlatMap transformation:


['Spark', 'is', 'awesome', 'Spark', 'is', 'cool']

## Transformación MapPartitions

*Devuelve un nuevo RDD aplicando una función a cada partición de este RDD*

In [16]:
x = sc.parallelize([1,2,3], 2)
def f(iterator): yield sum(iterator); yield 42
y = x.mapPartitions(f)
# glom() flattens elements on the same partition
print(x.glom().collect())
print(y.glom().collect())

[[1], [2, 3]]
[[1, 42], [5, 42]]


### Coalesce Transformation

*Devuelve un nuevo RDD reducido a un número menor de particiones*.

`coalesce(numPartitions, shuffle=False)`

In [17]:
x = sc.parallelize([1, 2, 3, 4, 5], 3)
y = x.coalesce(2)
print(x.glom().collect())
print(y.glom().collect())

[[1], [2, 3], [4, 5]]
[[1], [2, 3, 4, 5]]


### KeyBy Transformation

*Crea un RDD de pares, formando un par por cada elemento del RDD original. La clave del par se calcula a partir del valor mediante una función suministrada por el usuario.

In [18]:
x = sc.parallelize(['John', 'Fred', 'Anna', 'James'])
y = x.keyBy(lambda w: w[0])
print(y.collect())

[('J', 'John'), ('F', 'Fred'), ('A', 'Anna'), ('J', 'James')]


### Transformación PartitionBy

*Devuelve un nuevo RDD con el número de particiones especificado, colocando los elementos originales en la partición devuelta por una función suministrada por el usuario*

`partitionBy(numPartitions, partitioner=portable_hash)`

In [19]:
x = sc.parallelize([('J','James'),
                    ('F','Fred'),
                    ('A','Anna'),
                    ('J','John')], 3)

y = x.partitionBy(2, lambda w: 0 if w[0] < 'H' else 1)
print(x.glom().collect())
print(y.glom().collect())

[[('J', 'James')], [('F', 'Fred')], [('A', 'Anna'), ('J', 'John')]]


[[('F', 'Fred'), ('A', 'Anna')], [('J', 'James'), ('J', 'John')]]


### Transformación de la cremallera

*Devuelve un nuevo RDD que contiene pares cuya clave es el elemento del RDD original, y cuyo
valor es el elemento correspondiente de ese elemento (misma partición, mismo índice) en un segundo RDD*.

`zip(otroRDD)`

In [20]:
x = sc.parallelize([1, 2, 3])
y = x.map(lambda n:n*n)
z = x.zip(y)
print(z.collect())

[(1, 1), (2, 4), (3, 9)]


## Actions

<table class="table">
<tbody><tr><th>Action</th><th>Meaning</th></tr>
<tr>
  <td> <b>reduce</b>(<i>func</i>) </td>
  <td> Aggregate the elements of the dataset using a function <i>func</i> (which takes two arguments and returns one). The function should be commutative and associative so that it can be computed correctly in parallel. </td>
</tr>
<tr>
  <td> <b>collect</b>() </td>
  <td> Return all the elements of the dataset as an array at the driver program. This is usually useful after a filter or other operation that returns a sufficiently small subset of the data. </td>
</tr>
<tr>
  <td> <b>count</b>() </td>
  <td> Return the number of elements in the dataset. </td>
</tr>
<tr>
  <td> <b>first</b>() </td>
  <td> Return the first element of the dataset (similar to take(1)). </td>
</tr>
<tr>
  <td> <b>take</b>(<i>n</i>) </td>
  <td> Return an array with the first <i>n</i> elements of the dataset. </td>
</tr>
<tr>
  <td> <b>takeSample</b>(<i>withReplacement</i>, <i>num</i>, [<i>seed</i>]) </td>
  <td> Return an array with a random sample of <i>num</i> elements of the dataset, with or without replacement, optionally pre-specifying a random number generator seed.</td>
</tr>
<tr>
  <td> <b>takeOrdered</b>(<i>n</i>, <i>[ordering]</i>) </td>
  <td> Return the first <i>n</i> elements of the RDD using either their natural order or a custom comparator. </td>
</tr>
<tr>
  <td> <b>saveAsTextFile</b>(<i>path</i>) </td>
  <td> Write the elements of the dataset as a text file (or set of text files) in a given directory in the local filesystem, HDFS or any other Hadoop-supported file system. Spark will call toString on each element to convert it to a line of text in the file. </td>
</tr>
<tr>
  <td> <b>saveAsSequenceFile</b>(<i>path</i>) <br> (Java and Scala) </td>
  <td> Write the elements of the dataset as a Hadoop SequenceFile in a given path in the local filesystem, HDFS or any other Hadoop-supported file system. This is available on RDDs of key-value pairs that implement Hadoop's Writable interface. In Scala, it is also
   available on types that are implicitly convertible to Writable (Spark includes conversions for basic types like Int, Double, String, etc). </td>
</tr>
<tr>
  <td> <b>saveAsObjectFile</b>(<i>path</i>) <br> (Java and Scala) </td>
  <td> Write the elements of the dataset in a simple format using Java serialization, which can then be loaded using
    <code>SparkContext.objectFile()</code>. </td>
</tr>
<tr>
  <td> <b>countByKey</b>() <a name="CountByLink"></a> </td>
  <td> Only available on RDDs of type (K, V). Returns a hashmap of (K, Int) pairs with the count of each key. </td>
</tr>
<tr>
  <td> <b>foreach</b>(<i>func</i>) </td>
  <td> Run a function <i>func</i> on each element of the dataset. This is usually done for side effects such as updating an Accumulator or interacting with external storage systems.
  <br><b>Note</b>: modifying variables other than Accumulators outside of the <code>foreach()</code> may result in undefined behavior. See Understanding closures for more details.</td>
</tr>
</tbody></table>

### GetNumpartitions Action

*Retorna el número de particiones en RDD*

In [21]:
x = sc.parallelize([1,2,3], 2)
y = x.getNumPartitions()
print(x.glom().collect())
print(y)

[[1], [2, 3]]
2


In [22]:
sc.stop()